# <big>Transformer</big>

**Author: yidong jin**

**Date: 2023-5-21**

![pic1](png/pic1.png)
## 宏观上看transformer结构主要分为以下几个部分：
1. Embedding
    * token embedding
    * position encoding
2. Encoder
    * mutil-head attention
    * layer normalization and residual connect
    * feed forward layer
3. Decoder
    * masked mutil-head attention
    * layer normalization and residual connect
    * mutil-head attention
    * feed forward layer
4. output layer
    * linear layer
    * softmax

    
**整个网络的基本流程为输入先经过embedding被编码成向量，因为向量不带有位置信息，因此需要根据输入每句话中词的位置来编码一个位置向量，将这两个向量相加后输入编码器，编码器主要包含三部分，第一部分就是最核心的多头注意力模块，输入通过此模块得到不同向量之间的注意力系数，接着就是一个层归一化和残差连接，最后的feed forward layer其本质就是两个全连接层，解码器和编码器的不同之处主要有两个，一个是mask过的多头注意力模块，另一个是第二个多头注意力模块的输入有一部分直接来自于编码器，同时解码器需要输入每一个输入的真实标签来辅助训练，这种技术叫做teacher-forcing，这会显著的改善模型训练难度并加快收敛速度，最后再经过一个线性层和softmax函数就得到了最终的概率矩阵。接下来开始按顺序一步步搭建出transformer模型，let's start,coding is all you need!**

## embedding
**首先我们开始定义token embedding和position encoding，token embedding我们可以直接使用`torh.nn.Embedding`，position encoding则根据原论文中的公式进行编写。**

![pic3](png/pic3.png)

**在做embedding之前首先要明确我们传入的数据形式是什么样的，我们并不是直接把一句话的每个词都直接传入网络，而是会有一个vocabulary字典，每一个单词都对应一个唯一的索引，在传入网络前我们会将每个单词都mapping到其索引上。举个例子，现在有一句话“hello world!"，首先将其分离成单个字符存储在列表里得到[['hello','world','!']]，这里使用嵌套列表是因为通常我们一次输入网络不止一个句子，外层列表的长度即是batch size,接着将每一个词映射到vocabulary中，得到一个索引列表[[3,5,2]]，这个索引列表即是我们需要传入网络的输入。传入torch的embedding函数后函数会根据你的单词数量和每一个词需要映射的维度把每一句话都处理成一个维度为[seq_len,embedding_dim]的二维矩阵，这个二维矩阵的每一行即为一个词向量，词向量的维度取决于你的vocabulary有多大，你的字典里的词越多就需要越大的维度去覆盖所有的词。接着我们来看postion encoding的公式，其中pos代表每个词在句子中的位置，i代表词向量维度的索引，简单明了，只需要按照公式实现即可。**

In [17]:
import torch
import torch.nn as nn

class Token_Embedding(nn.Embedding):
    def __init__(self,voc_num,d_model):
        super().__init__(voc_num,d_model,padding_idx=0)

        
class Position_Encoder(nn.Module):
    def __init__(self,d_model):
        super().__init__()
        self.d_model = d_model
    
    def forward(self,x):
        batch,seq_len = x.size()
        # 位置编码不需要参与梯度更新
        pos_encoding = torch.zeros((seq_len,self.d_model),requires_grad=False)
        idx = torch.arange(0,seq_len).unsqueeze(dim=1)
        col_2_interval = torch.arange(0,self.d_model,2)
        # 遍历矩阵每一个值也可以但太low，完全没有利用矩阵运算的并行特性，不考虑使用
        # TODO:是否可以统一奇偶维度？
        if self.d_model % 2 == 0:
            pos_encoding[:,0::2] = torch.sin(idx / 10000 ** (col_2_interval/self.d_model)).float()
            pos_encoding[:,1::2] = torch.cos(idx / 10000 ** (col_2_interval/self.d_model)).float()
        else:
            pos_encoding[:,0::2] = torch.sin(idx / 10000 ** (col_2_interval/self.d_model)).float()
            pos_encoding[:,1::2] = torch.cos(idx / 10000 ** (col_2_interval/self.d_model))[:,:-1].float()
        return pos_encoding

    
class Embedding(nn.Module):
    def __init__(self,voc_num,d_model,p_drop):
        super().__init__()
        self.token_emb = Token_Embedding(voc_num,d_model)
        self.pos_emb = Position_Encoder(d_model)
        self.drop = nn.Dropout(p=p_drop)
        
    def forward(self,x):
        token_emb = self.token_emb(x)
        pos_emb = self.pos_emb(x)
        return self.drop(token_emb+pos_emb)

In [22]:
#embedding模块测试
inp = torch.LongTensor([[1,2,4,5],[2,3,1,8]])
emb = Embedding(10,4,0.1)
emb_out = emb(inp)
print(emb_out.shape)

torch.Size([2, 4, 4])


解释下这个矩阵每一个维度代表什么，第一维代表batch size即每次我们传入的句子数量，第二维代表每个句子的单词数，第三维代表编码成词向量的维度，我们输入了两个句子，每个句子四个词，定义使用三维的词向量进行编码，输出正确测试通过～

# mutil-head attention
**什么是注意力？我们都知道在很多的语言体系中都有多义词的概念，即一个词会有多种不同的含义，因此人类在翻译一段话中的每一个词时不会只单独关注当前词的意思，还需要结合上下文来明确该词的确切意思，因此我们的注意力需要进行分配，不仅聚焦在当前词上还需要聚焦在能够最大化帮助我们明确当前词意思的上下文信息上，当然人类可以自主做到这些，但模型不是人类，因此我们需要设计一种算法来让模型学会如何去自动的分配注意力让其能最准确的翻译每一个词，self-attention就此提出，该算法的基本思想是每翻译一个词时都会计算该词和这句话中其他的所有词的相关性，将相关性量化到0-1之间的权重后和每一个词向量做element-wise product后就可以使网络在学习时关注权重高的词向量。**

**那么首先的问题就是要搞清相关性是怎么计算的，我们的输入在经过embedding后已经变成了一个三维的矩阵，抛开batch size不看，我们单独取其中的一个二维矩阵，其本质就是一句话的编码矩阵，每一行就是一个单独的词向量，因此计算词之间的相关性就转变成了计算词向量之间的相关性，那么向量之间的相关性怎么计算呢？那就是向量的点积，点积可以表征两个向量的夹角也可以表示一个向量在另一个向量上的投影值，根据线代的知识，两个向量垂直即线性无关，此时向量的点积为0，因此使用点积值来表示相关性是很自然的事。**

![pic5](png/pic5.png)

**接下来就是self-attention的具体计算过程，其总体结构如上图所示，我们需要传入三个矩阵query，key和value，Q和K用来计算注意力系数，计算得到的系数经过缩放后输入softmax函数得到0-1之间的系数值，再和V相乘给每一个词向量分配相应的权重。Q,K,V均通过线性变换得到且线性变换的系数是可学习的，即让网络自己找出最适合的变换矩阵。**

![pic6](png/pic6.png)

**得到Q,K,V后就要开始计算每一个词向量和其他词向量的相关性了，前面说过矩阵的每一行都代表一个词向量，但如果两个向量要做点积的话就必须保证一个是行向量一个是列向量，因此只要将K矩阵做一个转置，行向量就变成了列向量，然后根据矩阵的运算规则，Q的第一行和K的每一列做点积运算得到输出矩阵的第一行，这样就得到了第一个词和其他每个词也包括它自己本身的相关值了，其它行同理。在观察这两个矩阵的运算过程我们会惊喜的发现矩阵的行和列的计算是同时进行的，我们通过一次矩阵运算就得到了每一个词和其他词的相关值，这就是矩阵并行计算的魅力。$QK^{T}$得到相关值后为什么又除了$\sqrt{d_{k}}$？那是因为每个词在经过embedding编码后都变成了均值为0，方差为1的独立随机分布词向量，Q和K每个词向量的维度都为$d_{k}$，$QK^{T}$后就得到一个$d_{k}xd_{k}$的方阵，因为词向量之间都是独立分布的，因此点积后得到的向量的均值和方差等于所有词向量的均值和方差之和，所以点积后的向量的均值和方差为0和$d_{k}$，由于$d_{k}$通常都会很大，这会导致向量里的值呈现一个非常大的数量级，如果直接传入softmax函数就会进入饱和区，而饱和区的梯度几乎为0，这样在反向传播时就会出现梯度消失问题而无法训练，所以除以$\sqrt{d_{k}}$使得每个词向量又回到均值0，方差1的分布。**

![pic7](png/pic7.png)

**得到相关值后输入softmax函数，需要注意的是这里的softmax函数是对矩阵的每一行做，使得每一行的和为1。**

![pic8](png/pic8.png)

**从softmax函数出来的矩阵即为我们需要的attention系数矩阵，矩阵的每一行即为当前词和其他词包括自身的attention系数，比如矩阵的第一行即为单词1分别和单词1，2，3，4的attention系数，因此根据矩阵乘法的运算规则，单词1的输出$Z_{1}$就为每个词的attention系数和对应词向量的线性加权。**

![pic9](png/pic9.png)

![pic10](png/pic10.png)

In [90]:
class Scaled_Dot_Product_Attention(nn.Module):
    def __init__(self):
        super().__init__()
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self,q,k,v,mask=None):
        # 输入为一个四维的矩阵，[batch,heads,seq_len,d_k or d_v]
        # 直接使用四维矩阵，充分利用矩阵并行运算
        batch,heads,seq_len,d = v.size()
        k_t = k.transpose(2,3)
        atten_score = (q @ k_t) / torch.sqrt(torch.tensor(d))
        
        if mask is not None:
            assert mask.shape == atten_score.shape,'mask shape {} is not equal to attention score shape {}.'.format(mask.shape,atten_score.shape)
            atten_score = atten_score.masked_fill(mask==0,-1e6) #用极小值填充使得softmax后值为0
        
        score = self.softmax(atten_score)
        value = score @ v
        # TODO:返回score便于后续实现可视化attention系数的heatmap
        return score,v
        


class Mutil_Head_Attention(nn.Module):
    def __init__(self,d_model,d_k,d_v,heads,mask=None,is_visual=False):
        super().__init__()
        self.q_w = nn.Linear(d_model,d_model)
        self.k_w = nn.Linear(d_model,d_model)
        self.v_w = nn.Linear(d_model,d_model)
        self.heads = heads
        self.d_k = d_k
        self.d_v = d_v
        self.mask = mask
        self.is_visual = is_visual
        self.attention = Scaled_Dot_Product_Attention()
        self.lin_concat = nn.Linear(d_model,d_model)
        
        
    def split_tensor(self,x):
        batch,seq_len,d_model = x.size()
        assert d_model // self.heads == self.d_k == self.d_v,'d_k mutilply heads not equal to d_model.'
        mutil_head_tensor = x.view(batch,seq_len,self.heads,d_model//self.heads).transpose(1,2)
        return mutil_head_tensor
    

        
    def forward(self,q,k,v):
        # shape[batch,seq_len,d_model]
        q,k,v = self.q_w(q),self.k_w(k),self.v_w(v)
        
        # 按heads数拆分成四维矩阵
        mutil_head_q,mutil_head_k,mutil_head_v = self.split_tensor(q),self.split_tensor(k),self.split_tensor(v)
        score,value = self.attention(mutil_head_q,mutil_head_k,mutil_head_v,mask=self.mask)
    
        # concat回原来的维度，因为是四维矩阵所以直接reshape就能搞定
        batch,heads,seq_len,d = value.size()
        out = value.transpose(1,2).contiguous().view(batch,seq_len,heads * d)
        out = self.lin_concat(out)
        if self.is_visual:
            return score,out
        return out

In [31]:
# 模块测试,输入和输出维度一致则通过
attention = Mutil_Head_Attention(4,2,2,2)
out = attention(emb_out,emb_out,emb_out)
print(out.shape)

torch.Size([2, 4, 4])


# layer normalization

![pic11.png](png/pic11.png)

**上图是来自何凯明论文中对于四种不同归一化方法的形象表示，通过这张图我们能很清楚的看出batch norm和layer norm之间的区别，batch norm是对一个batch内所有输入相同通道的数据做归一化，是一种纵向的归一化方法，而layer norm是对batch内每个输入数据单独做归一化，是一种横向的归一化方法，那么为什么transformer使用的是layer norm呢？因为针对多个语句输入，我们更关注的是每个语句内不同词之间的分布，而不是多个语句的许多词之间的分布，因此使用layer norm来进行归一化。layer norm的公式和batch norm一样都是减去均值除以标准差进行归一化，但要注意的是在标准化之后还加上了参数可学习的线性缩放，因为标准的归一化可能会将下层神经元的结果粗暴的调整到归一化区间，这会降低神经网络的表达能力。**

In [41]:
class Layer_Norm(nn.Module):
    def __init__(self,d_model,eps=1e-5):
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
    
    def forward(self,x):
        mean = x.mean(dim=-1,keepdim=True)
        # 为了和pytorch的官方实现保持一致，采用无偏估计计算方差
        var = x.var(dim=-1,unbiased=False,keepdim=True)
        out = (x - mean) / torch.sqrt(var + self.eps)
        out = self.alpha * out + self.beta
        return out

In [44]:
# 测试，和pytorch官方实现结果对比
a = torch.rand(1,3,3)
layer_norm = Layer_Norm(3)
out1 = layer_norm(a)
out2 = nn.LayerNorm(3)(a)
print(a)
print(out1)
print(out2)

tensor([[[0.2760, 0.8115, 0.1931],
         [0.7855, 0.7638, 0.2516],
         [0.8911, 0.5841, 0.9164]]])
tensor([[[-0.5505,  1.4033, -0.8528],
         [ 0.7505,  0.6627, -1.4132],
         [ 0.6217, -1.4106,  0.7889]]], grad_fn=<AddBackward0>)
tensor([[[-0.5505,  1.4033, -0.8528],
         [ 0.7505,  0.6627, -1.4132],
         [ 0.6217, -1.4106,  0.7889]]], grad_fn=<NativeLayerNormBackward0>)


结果完全相同，其实也没那么难嘛～

## feed forward layer
就是两个全连接层，第一个连接层扩充维度，第二个连接层压缩回原来的维度，输入和输出的维度相同，没什么可说的

In [45]:
class Feed_Forward(nn.Module):
    def __init__(self,d_model,d_ff,p_prob):
        super().__init__()
        self.layer1 = nn.Linear(d_model,d_ff)
        self.layer2 = nn.Linear(d_ff,d_model)
        self.drop = nn.Dropout(p=p_prob)
        self.relu = nn.ReLU()
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.drop(out)
        out = self.layer2(out)
        return out

In [46]:
# 模块测试
arr_tensor = torch.rand(1,3,3)
out = Feed_Forward(3,5,0.1)(arr_tensor)
print(out.shape)

torch.Size([1, 3, 3])


# encoder
**上面展示的是一层编码器层，encoder就是将编码器重复堆叠数层。**

In [80]:
class Encoder_Layer(nn.Module):
    def __init__(self,d_model,d_ff,heads,d_k,d_v,p_prob):
        super().__init__()
        self.attention = Mutil_Head_Attention(d_model,d_k,d_v,heads)
        self.norm1 = Layer_Norm(d_model)
        self.drop1 = nn.Dropout(p=p_prob)
        self.ffn = Feed_Forward(d_model,d_ff,p_prob)
        self.norm2 = Layer_Norm(d_model)
        self.drop2 = nn.Dropout(p=p_prob)
    
    def forward(self,x):
        identity_1 = x
        out = self.attention(x,x,x)
        out = self.norm1(torch.add(identity_1,self.drop1(out)))
        identity_2 = out
        out = self.ffn(out)
        output = self.norm2(torch.add(identity_2,self.drop2(out)))
        return output
    

class Encoder(nn.Module):
    def __init__(self,d_model,d_ff,heads,d_k,d_v,p_prob,n_layers):
        super().__init__()
        self.encoder = self.make_layers(d_model,d_ff,heads,d_k,d_v,p_prob,Encoder_Layer,n_layers)
        
    @staticmethod
    def make_layers(d_model,d_ff,heads,d_k,d_v,p_prob,layer,n_layers):
        layers = [layer(d_model,d_ff,heads,d_k,d_v,p_prob) for _ in range(n_layers)]
        return nn.Sequential(*layers)
    
    def forward(self,x):
        out = self.encoder(x)
        return out

In [81]:
# 测试
a = torch.rand(1,3,4)
out = Encoder(4,8,2,2,2,0.1,6)(a)
print(out.shape)

torch.Size([1, 3, 4])


# decoder
**decoder同理。**

In [82]:
class Decoder_layer(nn.Module):
    def __init__(self,d_model,d_ff,heads,d_k,d_v,p_prob,mask):
        super().__init__()
        self.mask_attention = Mutil_Head_Attention(d_model,d_k,d_v,heads,mask=mask)
        self.norm1 = Layer_Norm(d_model)
        self.drop1 = nn.Dropout(p=p_prob)
        self.attention = Mutil_Head_Attention(d_model,d_k,d_v,heads)
        self.norm2 = Layer_Norm(d_model)
        self.drop2 = nn.Dropout(p=p_prob)
        self.ffn = Feed_Forward(d_model,d_ff,p_prob)
        self.norm3 = Layer_Norm(d_model)
        self.drop3 = nn.Dropout(p=p_prob)
        
    def forward(self,x_encoder,x_decoder):
        identity_dec = x_decoder
        out = self.mask_attention(x_decoder,x_decoder,x_decoder)
        out = self.norm1(torch.add(identity_dec,self.drop1(out)))
        identity_1 = out
        out = self.attention(out,x_encoder,x_encoder)
        out = self.norm2(torch.add(identity_1,self.drop2(out)))
        identity_2 = out
        out = self.ffn(out)
        output = self.norm3(torch.add(identity_2,self.drop3(out)))
        return output
    
    
class Decoder(nn.Module):
    def __init__(self,d_model,d_ff,heads,d_k,d_v,p_prob,mask,n_layers):
        super().__init__()
        self.decoder = nn.ModuleList([Decoder_layer(d_model,d_ff,heads,d_k,d_v,p_prob,mask) for _ in range(n_layers)])
        
    def forward(self,x_enc,x_dec):
        for layer in self.decoder:
            x_dec = layer(x_enc,x_dec)
        
        return x_dec

In [84]:
# 测试
mask = torch.tril(torch.ones(1,2,3,3))
inp1 = torch.rand(1,3,6)
inp2 = torch.rand(1,3,6)
out = Decoder(6,8,2,3,3,0.1,mask,6)(inp1,inp2)
print(out.shape)

torch.Size([1, 3, 6])


## transformer
**终于定义完了所有的需要用到的模块，现在只需要将它们按照一定规则组合起来就完成啦，这也是我最喜欢pytorch框架的一点，就像小时候玩的乐高积木一样，只要将所有的积木找齐一层层搭起来就能得到你想要的结构。**

In [86]:
class Transformer(nn.Module):
    def __init__(self,voc_num,d_model,d_ff,heads,d_k,d_v,p_prob,mask,enc_layers,dec_layers):
        super().__init__()
        self.emb = Embedding(voc_num,d_model,p_prob)
        self.Encoder = Encoder(d_model,d_ff,heads,d_k,d_v,p_prob,enc_layers)
        self.Decoder = Decoder(d_model,d_ff,heads,d_k,d_v,p_prob,mask,dec_layers)
        self.linear = nn.Linear(d_model,voc_num)
        self.softmax = nn.Softmax(dim=-1)
     
    
    def forward(self,x,y):
        x_enc = self.emb(x)
        x_dec = self.emb(y)
        out = self.Encoder(x_enc)
        identity_enc = out
        out = self.Decoder(identity_enc,x_dec)
        out = self.linear(out)
        output = self.softmax(out)
        return output

In [91]:
# 测试
x = torch.LongTensor([[1,3,4,5,2,2],[2,3,5,6,7,9],[4,3,7,5,2,8]])
y = torch.LongTensor([[1,3,4,5,2,2],[2,3,5,6,7,9],[4,3,7,5,2,8]])
voc_num = 10
d_model = 6
d_ff = 10
heads = 2
d_k = 3
d_v = 3
p_prob = 0.1
mask = torch.tril(torch.ones(3,2,6,6))
enc_layers = 6
dec_layers = 6
transformer = Transformer(voc_num,d_model,d_ff,heads,d_k,d_v,p_prob,mask,enc_layers,dec_layers) 
output = transformer(x,y)
print(output.shape)

torch.Size([3, 6, 10])


__整个模型搭建完毕，希望能通过整个流程使的看文档的人对transformer架构有一个更深刻的认识，keep learning is all we need😀__